<a href="https://colab.research.google.com/github/alon-a/NAYA-Project-1/blob/main/Drone_Route_Optimize_Project_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the first project in the NAYA College Data-Scientist course by Alon Avramson.

This project is part of a greater solution: A Drone-Shipping System using Gas-Stations Roofs with route optimization.
The current project will select the map library, translate gas-stations locations to addresses and place a marker on the map for this location. 

# Review the optional MAP libraries

First let's look for a map library (Yahoo search "pyton maps" or "python geo map library"):
- [Sharpfiles](https://towardsdatascience.com/mapping-geograph-data-in-python-610a963d2d7f) PyShp. Developed and regulated by Esri as a (mostly) open specification, the shapefile format spatially describes geometries as either ‘points’, ‘polylines’, or ‘polygons’. In OpenStreetMap terms these can be considered as ‘nodes’, ‘ways’ and ‘closed ways’, respectively. Each geometry has a set of associated attributes. Broadly speaking these are a bit like OSM’s tags.
- [GeoPandas](https://towardsdatascience.com/mapping-geograph-data-in-python-610a963d2d7f) is an open source project to make working with geospatial data in python easier. [GeoPandas](https://geopandas.org/) 
- [Mapping with Matplotlib, Pandas, Geopandas and Basemap in Python ](https://towardsdatascience.com/mapping-with-matplotlib-pandas-geopandas-and-basemap-in-python-d11b57ab5dac)
- Basemap
Refining the Yahoo search: python disppaly a marker on geo maps 
- [Folium ast ](https://stackoverflow.com/questions/46560309/print-a-map-with-markers-and-text-details-with-python) StackOverflow ex.
- Folium - [Map with markers with Python and Folium](https://www.python-graph-gallery.com/312-add-markers-on-folium-map) Seems like the right choice for this project, it is easy to use, and light-weight
- [Interactive map with Python and Folium](https://www.python-graph-gallery.com/288-map-background-with-folium)
- [Show your Data in a Google Map with Python](https://thedatafrog.com/en/articles/show-data-google-map-python/) Requires Google API keys (credit card)
- With Folium and Pandas [Generating Maps with Python: “Maps with Markers”-Part 2](https://medium.com/analytics-vidhya/generating-maps-with-python-maps-with-markers-part-2-2e291d987821)
- [Map With Plotly, GeoPands, Basemap ](https://plotly.com/python/scatter-plots-on-maps/)
- [OpenStreetMap](https://towardsdatascience.com/how-safe-are-the-streets-of-santiago-e01ba483ce4b) may be useful later for extracting roads 

Summary:
- Folium - is my choice, simple, interactive, lightweight
- Google Maps - complex integration
- GeoPandas - Static maps
- Basemaps - resulting files are too heavy, resulting low interactivity and long processing time
- Sharpfiles - might be used later to add shapes on the map

# Preparation

Lets' look for translation of addresses to Geo-Lcation (Lat, Lon)
- Manual [Get Lat Long from Address](https://www.latlong.net/convert-address-to-lat-long.html)
- Geopy P[ython’s geocoding — Convert a list of addresses into a map](https://towardsdatascience.com/pythons-geocoding-convert-a-list-of-addresses-into-a-map-f522ef513fd6)
- GeoCode Address [GeoLOcation IP Address](https://geolocation.com/en_US)
- Geopy provides a class for popular mapping services. Nominatim is the service behind the popular OpenStreetMap that allows you to geocode for free. But you should comply with the usage policies in order to allow everyone to use it:     limit your requests to a single thread
    limited to 1 machine only, no distributed scripts (including multiple Amazon EC2 instances or similar)
    Results must be cached on your side. Clients sending repeatedly the same query may be classified as faulty and blocked.
- Nominatim 

[Python’s geocoding — Convert a list of addresses into a map](https://towardsdatascience.com/pythons-geocoding-convert-a-list-of-addresses-into-a-map-f522ef513fd6) This is a full example that will be used a skeleton and starting point for my project

create a pandas dataFrame - will be implemented after learning Pandas

Translate a text dats with addresses to a DataFrame with Lon Lat columns is a good start for any map presentation
- data = """Name,Address
EU,"Rue de la Loi/Wetstraat 175, Brussel, Belgium"
Apple,"1 Apple Park Way, Cupertino, CA"
Google,"1600 Amphitheatre Parkway Mountain View, CA 94043"
UN,"760 United Nations Plaza; Manhattan, New York City"
"""

[How to Get Geolocation in Python ](https://www.thepythoncode.com/article/get-geolocation-in-python) This is yet another simple example that will be used as skeleton in this project

In [1]:
# !pip install geopy

Getting Latitude and Longitude from an Address (Geocoding)

In [2]:
# pip install pprintpp
from geopy.geocoders import Nominatim
import time # since he Nominatim Usage Policy, requires  to use a maximum of 1 request per second, we may need to use sleep(1)
#from pprint import pprint # https://docs.python.org/3/library/pprint.html

In [3]:
# instantiate a new Nominatim client
app = Nominatim(user_agent="tutorial")

Checking the results with: https://www.latlong.net/convert-address-to-lat-long.html results look good

Now that I know how to create a geo-location from an address let's put it on a map and present it with a marker


# Let's find out the avaiable data for the list of Gas Stations in Israel



לפני זה נבדוק את אפשר להכניס כתובת בעברית על מנת לוודא שהחיפוש כתובות יכול להיות גם בעיברית



Yahoo search: רשימת כתובות של תחנות דלק בישראל

- http://delek4u.info/
- https://www.gov.il/he/Departments/General/gas_stations

https://pythonbasics.org/read-excel/ Read Excel files

# The project

Relevnt links

Get the stations file from the internet and save it locally as fuel_stations.xlsx. This file cintains more than 1,200 stations. It is  required only if you want to re-create the data from scratch. If not, the application will look for a pickled file on the local default location (.content) enerated from the excel file. The pickled file includes about 370 stations. If the pickled file is not found the application uses a short list of 25 station. 
https://www.gov.il/he/Departments/General/gas_stations

Data preparation

In [4]:
!pip install geopy
!pip install pprintpp

In [6]:
# Get the stations file from the internet and save it locally as fuel_stations.xlsx
# https://www.gov.il/he/Departments/General/gas_stations
import requests, os
import http.client

http.client.HTTPConnection._http_vsn = 10
http.client.HTTPConnection._http_vsn_str = 'HTTP/1.0'

url = 'https://www.gov.il/BlobFolder/generalpage/gas_stations/he/fuel_station_by_number2.xlsx'
r = requests.get(url, allow_redirects=True)

#https://www.pythonexcel.com/ 

#import openpyxl
#mywb = openpyxl.load_workbook('filetest.xlsx')
#sheet = mywb.active
#sheet.title = 'Working on Save as'
#mywb.save('example_filetest.xlsx')

if (r.ok):

  file_saved = 'fuel_stations.xlsx'

  print("Downloading...")
  with open(file_saved, 'wb') as output:
      output.write(r.content)
  print(f"Done! {file_saved} is ready")

else:
  print(f'File could not be downloaded, {(r.reason)} , file returned code: {r.status_code} ')
  print(r.url)


File could not be downloaded, Not Found , file returned code: 404 
https://www.gov.il/BlobFolder/generalpage/gas_stations/he/fuel_station_by_number2.xlsx


In [7]:
#getting back to the manually donwloaded file until downlad issues resolved
import xlrd
file_saved = 'fuel_station_by_number1.xlsx' 
gas_station_file_exist = True
if os.path.exists(file_saved):
  print('Reading the file...')
  wb = xlrd.open_workbook(file_saved)
  sh = wb.sheet_by_index(0)
  print('Reading the file...')
  sh
else:
  gas_station_file_exist = False
  print(file_saved, ' does not exist on local disk please check')

fuel_station_by_number1.xlsx  does not exist on local disk please check


In [8]:
# Extract the station's addresses and names into a list
if (gas_station_file_exist):
  stations_list = []
  for rx in range (10): #(sh.nrows): # using the first 10 lines as test, chane to sh.nrows for all rows
      station_name = (sh.cell(rx,1).value)
      station_location =  (sh.cell(rx,3).value)
      stations_list.append([station_name,station_location])
      pass
  #stations_list
  len(stations_list)

In [9]:
# import the libraries to create names and locations for the map
from geopy.geocoders import Nominatim
import time # since he Nominatim Usage Policy, requires  to use a maximum of 1 request per second, we may need to use sleep(1)
from pprint import pprint # https://docs.python.org/3/library/pprint.html

In [10]:
# initializing the Nominatim project
app = Nominatim(user_agent="tutorial")

Prepare the initial location for the   map

In [11]:
address = "ראש פינה 26, פתח תקווה , ישראל"
location = app.geocode(address).raw
# print raw data
#print(location)
#pprint(location) #checking the contribution of pprint over standard print
latitude = location["lat"]
longitude = location["lon"]
print(f"Lat: {latitude}, Lon: {longitude}")

Lat: 32.10405555, Lon: 34.87346514896548


In [12]:
# Verify that a map can be generated and dispalyed with the defoned initial location 
# import the library
import folium

# Make a map with deep zoom
m = folium.Map(location=[float(latitude),float(longitude) ], tiles="OpenStreetMap", zoom_start=15)
# Add the Marker
folium.Marker(
      location=[float(latitude),float(longitude)],
      popup="Inital Address",
   ).add_to(m)

#Show the map
m

In [13]:
#Save the file as HTML
m.save('My Initial Address Map.html')

Translate the list of addresses to geo-code loactions

In [14]:
# Translate the list of addresses to geo-codes locations (Lat,Lon)
stations_list_geocoded = []
if (gas_station_file_exist):
  continue_the_run = input('This process may take about an hour, are sure you want to proceed? type YES to continue  ')
  if continue_the_run != 'YES':
    pass
  else:
    for i in range(len(stations_list)):
      #print('Name: ', stations_list[i][0], 'Address: ',stations_list[i][1] )
      station_name    = stations_list[i][0]
      station_address = stations_list[i][1]
      time.sleep(1)
      print(station_name, station_address)
      try:
        geo_location = app.geocode(station_address).raw
        latitude = geo_location["lat"]
        longitude = geo_location["lon"]
        stations_list_geocoded.append([station_name,station_address, latitude, longitude  ])
        folium.Marker(
          location=[float(latitude),float(longitude)],
        popup=station_name,
      ).add_to(m)
      except:
        print('Could not find geolocation for: ', station_address) 
  m

In [15]:
#stations_list_geocoded
print(f'The list of geo-coded gas stations includes {len(stations_list_geocoded)} stations')

The list of geo-coded gas stations includes 0 stations


Saving the list of gas stations that are already geo-coded

In [16]:
# In this case, since the creation of the stations_list_geocoded took a long time
# to create, I save a pickle file in the name of"stations_list_geocoded_new.pkl"
# but then loads another file "stations_list_geocoded.pkl" which was generated and saved
if (len(stations_list_geocoded) == 0):
  print('Nothing to save, the list is empty')
else: 
  import pickle
  myvar = stations_list_geocoded
  print('Saving the stations_list_geocoded')
  with open('stations_list_geocoded_new.pkl', 'wb') as file:  
    # A new file will be created
    pickle.dump(myvar, file)
    print('Done')

Nothing to save, the list is empty


# Loading the list of gas stations that are already geo-coded

In [17]:
import ipywidgets as widgets
from IPython.display import display
button1 = widgets.Button(description="YES")
button2 = widgets.Button(description="NO")

def YES_click(x):
    print("YES clicked")
    
def NO_click(x):
    print("NO clicked")

button1.on_click(YES_click)
button2.on_click(NO_click)

#display(button1, button2)

# The geo-coded file was stored on a shared Dropbox location
# https://www.dropbox.com/s/lb2hqr2ezn11vbk/stations_list_geocoded.pkl?dl=0
import requests, os
import http.client

http.client.HTTPConnection._http_vsn = 10
http.client.HTTPConnection._http_vsn_str = 'HTTP/1.0'

url = 'https://www.dropbox.com/s/lb2hqr2ezn11vbk/stations_list_geocoded.pkl?dl=0'
r = requests.get(url, allow_redirects=True)

# https://www.w3schools.com/PYTHON/ref_requests_response.asp
if (r.ok):
  geo_coded_pickled_file_saved = 'stations_list_geocoded.pkl'

  if os.path.exists(pickled_file): 
    ans= input('File already exsist, do you want to override? YES/NO: ')
    if ( ans == 'YES'):
      print("Downloading...")
      with open(geo_coded_pickled_file_saved, 'wb') as output:
        output.write(r.content)
      print(f"Done! {geo_coded_pickled_file_saved} is overiiten and ready")
    else:
      print('File was not changed')
      pass
  else:
    print("Downloading...")
    with open(geo_coded_pickled_file_saved, 'wb') as output:
      output.write(r.content)
    print(f"Done! a new file {geo_coded_pickled_file_saved} is ready")
else:
  print(f' File could not be downloaded, {(r.reason)} , file returned code: {r.status_code} ')
  print(r.url)
  print(r.content)


 File could not be downloaded, Bad Request , file returned code: 400 
https://www.dropbox.com/s/lb2hqr2ezn11vbk/stations_list_geocoded.pkl?dl=0
b'<!DOCTYPE html>\n<html>\n<head><meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n<title>Dropbox</title>\n<link href="https://cfl.dropboxstatic.com/static/css/error.css" rel="stylesheet" type="text/css"/>\n<link rel="shortcut icon" href="https://cfl.dropboxstatic.com/static/images/favicon.ico"/>\n\n</head>\n<body>\n<div class="figure">\n<img src="https://cfl.dropboxstatic.com/static/images/illustration_catalog/sickbox-illo_m1.png" srcset="https://cfl.dropboxstatic.com/static/images/illustration_catalog/sickbox-illo_m1@2x.png 2x" alt="Error: 5xx"/>\n</div>\n<div id="errorbox">\n<h1>Error</h1>Something went wrong. Don\'t worry, your files are still safe and the Dropboxers have been notified. Check out our <a href="https://www.dropbox.com/help">Help Center</a> and <a href="https://forums.dropbox.com">forums</a> for help, or hea

In [18]:
import pickle
stations_list_geocoded_loaded =[]
pickled_file = "stations_list_geocoded.pkls"
if os.path.exists(pickled_file): 
  print('Loading the stations_list_geocoded')
  with open(pickled_file, "rb") as f:
    stations_list_geocoded_loaded = pickle.load(f)
    print('Done')
  #stations_list_geocoded_loaded
else:
  print(f'{pickled_file} file does not exist on local disk')
  ans = input('There is a default shot list defined, would you like to use it? YES/NO: ')
  if (ans == 'YES'):
    stations_list_geocoded_loaded = [['אדגו', 'זלמן שזר 1 חדרה', '32.4353872', '34.9118045'],
 ['אלונים', 'צומת אלונים', '32.70921', '35.142188'],
 ['אשל אברהם', 'באר שבע', '31.2457442', '34.7925181'],
 ['מיקה וולפסון', 'הלוחמים 1 חולון', '32.034701299999995', '34.77136495'],
 ['הסיבים', 'הסיבים 35 פ"ת', '32.0923047', '34.8572837'],
 ['הר תמיר', 'יעקב אליאב 1 גבעת שאול י-ם', '31.79496315', '35.18778583710159'],
 ['השבעה', 'דרך ירושלים 20 אזור', '31.525315', '34.603912'],
 ['טייבה', 'צומת השרון', '32.323644', '34.902989'],
 ['יאסין', 'באקה אל גרבייה', '32.4197144', '35.0428311'],
 ['דלק ירכא', 'ירכא', '32.9548871', '35.2087895'],
 ['דורון כפר סבא', 'נתיב האבות 1 כפר סבא', '32.1751957', '34.8911412'],
 ['אורות', 'דרך בר יהודה 34 חיפה', '32.7902739', '35.0266879'],
 ['סונול הבנים', 'הרצל 6 רמלה', '31.9349523', '34.8673488'],
 ['שעריים', 'הרצל 68 רחובות', '31.8937495', '34.8117132'],
 ['גבעונים', 'צומת גבעת זאב', '31.8705007', '35.1774813'],
 ['גל צומת הערבה', 'צומת הערבה', '30.987586', '35.308952'],
 ['מירון', 'צומת מירון-צפת', '32.987689', '35.442719'],
 ['גן לונדון', 'הירקון 83\n תל אביב', '32.0777722', '34.76685'],
 ['געתון', 'כביש עכו נהריה', '33.0476412', '35.1407717'],
 ['דרור זקס', 'צומת דרור', '32.257813', '34.900012'],
 ['נוה יעקב', 'שכונת נווה יעקב ירושלים', '32.0502416', '34.7854123'],
 ['מבואות דליה', 'דלית אל כרמל', '32.6922393', '35.0482785'],
 ['פארק אפק', 'כפר קאסם 49940', '32.1139357', '34.9760002'],
 ['פוליבה', 'רמלה', '31.9279988', '34.8623473'],
 ['הנביאים הדר חיפה', 'לבונטין 3 חיפה 33303', '32.8132268', '34.9934915']]
  print(f'The default list was loaded successfully')

print(f' The list of geo-coded stations includes: {len(stations_list_geocoded_loaded)} lines')

stations_list_geocoded.pkls file does not exist on local disk
There is a default shot list defined, would you like to use it? YES/NO: YES
The default list was loaded successfully
 The list of geo-coded stations includes: 25 lines


In [19]:
print(f' The list of geo-coded stations includes: {len(stations_list_geocoded_loaded)} lines')

 The list of geo-coded stations includes: 25 lines


In [20]:
# View the list 
#stations_list_geocoded_loaded

# Creating the functions required for creating a new map and adding stations names and locations

Create initial map object

In [21]:
def create_initial_map_location(station_name, lat, lon):
  # import the library
  import folium
  # Make a map with deep zoom
  m1 = folium.Map(location=[float(lat),float(lon) ], tiles="OpenStreetMap", zoom_start=15)
  # Add the Marker
  folium.Marker(
      location=[float(lat),float(lon)],
      popup=station_name,
  ).add_to(m1)
  print('A new map object was created')
  # Show the map
  return(m1)


In [22]:
# Check the function
station_map = create_initial_map_location('NEW STATION', 32.10405558, 34.87346514896540)

A new map object was created


In [23]:
# Show the map
station_map

Check geo-code for address

In [24]:
def check_geocode_for_address(address_string):
  import geopy
  from geopy.geocoders import Nominatim
  import time
  # instantiate a new Nominatim client
  app = Nominatim(user_agent="tutorial")
  time.sleep(1) # as the Nominatim requires minimum of 1 sec between requests
  try:
      geo_location = app.geocode(address_string).raw
      latitude = geo_location["lat"]
      longitude = geo_location["lon"]
      return(['OK',longitude,longitude])
  except:
      #print('Could not find geolocation for: ', address_string) 
      return(['NOK',address_string])

In [25]:
# Check the function - positive
check_geocode_for_address('דרך ארץ 2 חריש') # expect positive answer

['OK', '35.0383241', '35.0383241']

In [26]:
# Check the function - causal
check_geocode_for_address('זכרון יעקב 25 פתח תקווה ישראל')

['OK', '34.8771646', '34.8771646']

In [27]:
# Check the function - negative
check_geocode_for_address('המתכנן 10 א.ת. דרומי אשקלון') #expect negative answer

['NOK', 'המתכנן 10 א.ת. דרומי אשקלון']

Add a marker for a station with lat-lon to a map

In [28]:
def add_station_name_lat_lon_to_map(station_name, lat, lon, m1):
  # import the library
  import folium
  # Make a map with deep zoom
  #m1 = folium.Map(location=[float(lat),float(lon) ], tiles="OpenStreetMap", zoom_start=15)
  # Add the Marker
  folium.Marker(
      location=[float(lat),float(lon)],
      popup=station_name,
  ).add_to(m1)
  print(station_name, 'Added on ',lat,lon )
  # Show the map
  return(m1)


In [29]:
# Check funtcion
station_map = add_station_name_lat_lon_to_map('NEWER STATION', 32.103055, 34.872465148965, station_map)

NEWER STATION Added on  32.103055 34.872465148965


In [30]:
# Show the map
station_map

Add a marker for a station with address to the map

In [31]:
def add_station_name_address_to_map(station_name, address_string, m1):
  if check_geocode_for_address(address_string)[0] == 'OK':
    lat = check_geocode_for_address(address_string)[1]
    lon = check_geocode_for_address(address_string)[2]
    m1=add_station_name_lat_lon_to_map(station_name,lat ,lon,m1 )
    print(f'{station_name} on address: {address_string}  added successfully to the map {m1} on lat: {lat} lon: {lon})')
    return(m1)
  elif check_geocode_for_address(address_string)[0] == 'NOK':
    print(f'{address_string}  could not be geocoded')
    return(m1)

In [32]:
# Check the function
station_map = add_station_name_address_to_map('נקודה שלישית',('זכרון יעקב 25 פתח תקווה ישראל'),station_map) # expect positive answer

נקודה שלישית Added on  34.8771646 34.8771646
נקודה שלישית on address: זכרון יעקב 25 פתח תקווה ישראל  added successfully to the map <folium.folium.Map object at 0x7fe15fc649d0> on lat: 34.8771646 lon: 34.8771646)


In [33]:
# Show the map
# Note: here we see a successful address case but the lat,lon are abviously wrong
station_map

In [34]:
# Check another location
station_map = add_station_name_address_to_map('עוד','המתכנן 10 א.ת. דרומי אשקלון', station_map) #expect negative answer

המתכנן 10 א.ת. דרומי אשקלון  could not be geocoded


Populate the map with the loaded list of geo-coded stations


In [35]:
def populate_map_with_geo_coded_stations(m1):
  if (len(stations_list_geocoded_loaded) == 0):
    print(f'List is empty, nothing new to show')
    return(m1)
  else:
    for i in range(len(stations_list_geocoded_loaded)):
      #print(stations_list_geocoded_loaded[i])
      station_name = stations_list_geocoded_loaded[i][0]
      lat = stations_list_geocoded_loaded[i][2]
      lon = stations_list_geocoded_loaded[i][3]
      add_station_name_lat_lon_to_map(station_name, lat, lon, m1)
      print(f'{stations_list_geocoded_loaded[i]} marker was added to the map')
    return(m1)

In [36]:
station_map = populate_map_with_geo_coded_stations(station_map)

אדגו Added on  32.4353872 34.9118045
['אדגו', 'זלמן שזר 1 חדרה', '32.4353872', '34.9118045'] marker was added to the map
אלונים Added on  32.70921 35.142188
['אלונים', 'צומת אלונים', '32.70921', '35.142188'] marker was added to the map
אשל אברהם Added on  31.2457442 34.7925181
['אשל אברהם', 'באר שבע', '31.2457442', '34.7925181'] marker was added to the map
מיקה וולפסון Added on  32.034701299999995 34.77136495
['מיקה וולפסון', 'הלוחמים 1 חולון', '32.034701299999995', '34.77136495'] marker was added to the map
הסיבים Added on  32.0923047 34.8572837
['הסיבים', 'הסיבים 35 פ"ת', '32.0923047', '34.8572837'] marker was added to the map
הר תמיר Added on  31.79496315 35.18778583710159
['הר תמיר', 'יעקב אליאב 1 גבעת שאול י-ם', '31.79496315', '35.18778583710159'] marker was added to the map
השבעה Added on  31.525315 34.603912
['השבעה', 'דרך ירושלים 20 אזור', '31.525315', '34.603912'] marker was added to the map
טייבה Added on  32.323644 34.902989
['טייבה', 'צומת השרון', '32.323644', '34.902989'] 

In [37]:
# Show the map
station_map

In [38]:
station_map.save('Stations Map.html')